<a href="https://colab.research.google.com/github/EgwuSamuel/EgwuSamuel/blob/main/maize%20virus%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install specific versions of ultralytics and tensorflow
!pip install ultralytics tensorflow

# Install ipywidgets for interactive widgets in Jupyter notebooks
!pip install ipywidgets

# Upgrade ultralytics to the latest version
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.3/915.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
# Step 1: Download the dataset from Kaggle
!pip install -q kaggle

# Step 2: Set up Kaggle API Key (upload kaggle.json here in Colab for access)
from google.colab import files
files.upload()  # Upload kaggle.json file

# Move kaggle.json to the correct directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the dataset
!kaggle datasets download -d smaranjitghose/corn-or-maize-leaf-disease-dataset
!unzip corn-or-maize-leaf-disease-dataset.zip -d maize_leaf_data

# Download and unzip the dataset
!kaggle datasets download -d carlangelopamplona/corn-leaf-private-testing-v3
!unzip corn-leaf-private-testing-v3.zip -d maize_leaf_data_testing

Streaming output truncated to the last 5000 lines.
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1062).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1063).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1064).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1065).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1066).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1067).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1068).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1069).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (107).jpg  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1070).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1071).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1072).JPG  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1073).jpg  
  inflating: maize_leaf_data/data/Blight/Corn_Blight (1074).JPG  
  inflating: maize_leaf_da

In [3]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split

# Define paths
data_folder = '/content/maize_leaf_data/data'  # Replace with actual path
testing_folder_source = '/content/maize_leaf_data_testing'  # Replace with actual path

# Define train, validation, and test folders
base_dir = "/content/data/working/custom"
train_folder = os.path.join(base_dir, "train")
val_folder = os.path.join(base_dir, "val")
test_folder = os.path.join(base_dir, "test")
extra_test_folder = "/content/data/working/test"  # Duplicate test set

# Ensure all folders exist
for folder in [train_folder, val_folder, test_folder, extra_test_folder]:
    os.makedirs(folder, exist_ok=True)

# Function to copy files with error handling
def copy_file(src, dest):
    try:
        shutil.copy2(src, dest)
    except Exception as e:
        print(f"Error copying {src} → {dest}: {e}")

# Function to copy dataset
def copy_dataset(src_folder, dest_folder):
    for item in os.listdir(src_folder):
        src_path = os.path.join(src_folder, item)
        dest_path = os.path.join(dest_folder, item)
        if os.path.isdir(src_path):
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
        else:
            copy_file(src_path, dest_path)

# Copy validation dataset from testing source
copy_dataset(testing_folder_source, val_folder)

# Retrieve class folders
if not os.path.exists(data_folder):
    raise FileNotFoundError(f"Data folder not found: {data_folder}")

subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]

for subfolder in subfolders:
    class_name = os.path.basename(subfolder)
    files = [f.path for f in os.scandir(subfolder) if f.is_file()]

    # Limit to 570 samples per class
    files = files[:570] if len(files) > 570 else files

    # Train (80%), Validation (10%), Test (10%) Split
    train_files, temp_files = train_test_split(files, test_size=0.2, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

    # Create class-specific subdirectories
    for folder in [train_folder, val_folder, test_folder, extra_test_folder]:
        os.makedirs(os.path.join(folder, class_name), exist_ok=True)

    # Function to handle parallel file copying
    def parallel_copy(file_list, destination):
        dest_folder = os.path.join(destination, class_name)
        with ThreadPoolExecutor() as executor:
            executor.map(lambda f: copy_file(f, dest_folder), file_list)

    # Copy files in parallel
    parallel_copy(train_files, train_folder)
    parallel_copy(val_files, val_folder)
    parallel_copy(test_files, test_folder)
    parallel_copy(test_files, extra_test_folder)  # Duplicate test set

print("✅ Data successfully split into train (80%), validation (10%), and test (10%) sets with 570 samples per class.")


✅ Data successfully split into train (80%), validation (10%), and test (10%) sets with 570 samples per class.


In [4]:
# Install the wandb (Weights & Biases) library for experiment tracking
!pip install wandb

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
import wandb

# Initialize wandb for logging (remove 'mode="disabled"' for active tracking)
wandb.init(project="YOLO_Optimization", name="YOLO_Finetune", mode="disabled")

# Load a pretrained YOLO model with transfer learning
model = YOLO('yolo11n-cls.pt')  # Load a lightweight YOLO model for classification

# Set training parameters
train_params = {
    "data": "/content/data/working/custom",  # Path to the dataset
    "epochs": 50,  # Increase epochs for better convergence
    "batch": 32,  # Larger batch size for stable gradients
    "imgsz": 640,  # Higher resolution for better feature extraction
    "optimizer": "AdamW",  # AdamW improves optimization over standard Adam
    "lr0": 1e-3,  # Initial learning rate
    "lrf": 0.1,  # Learning rate final multiplier
    "cos_lr": True,  # Cosine annealing learning rate scheduler
    "patience": 5,  # Early stopping to prevent overfitting
    "augment": True,  # Enable data augmentation for better generalization
    "flipud": 0.5,  # Apply vertical flip augmentation
    "fliplr": 0.5,  # Apply horizontal flip augmentation
    "hsv_h": 0.015,  # Hue augmentation
    "hsv_s": 0.7,  # Saturation augmentation
    "hsv_v": 0.4,  # Brightness augmentation

}

# Train the model with optimized settings
model.train(**train_params)

# Validate model performance
metrics = model.val()

# Print the results for evaluation
print(metrics)


Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data/working/custom, epochs=50, time=None, patience=5, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/data/working/custom/train... 1824 images, 0 corrupt: 100%|██████████| 1824/1824 [00:00<00:00, 3193.36it/s]

train: New cache created: /content/data/working/custom/train.cache



val: Scanning /content/data/working/custom/val... 1097 images, 0 corrupt: 100%|██████████| 1097/1097 [00:00<00:00, 3950.17it/s]

val: New cache created: /content/data/working/custom/val.cache


...


      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [04:03<00:00, 13.51s/it]

                   all      0.857          1



      Epoch    GPU_mem       loss  Instances       Size


       5/50         0G     0.1974         32        640:   4%|▎         | 2/57 [00:40<18:35, 20.28s/it]